In [40]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb 
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score
import requests
import urlparse
import os
import pickle
from bs4 import BeautifulSoup
import urllib2
import re
from datetime import datetime
from dateutil.parser import *
import warnings; warnings.simplefilter('ignore')

# Clean Revenue and Budget

In [14]:
md = pd. read_csv('../data/movies_metadata.csv')
nsfw_data = pd.read_csv('../data/nsfw_score.csv')
credits = pd.read_csv('../data/credits.csv')
keywords = pd.read_csv('../data/keywords.csv')
nsfw_data.columns = ['id', 'nsfw_score']
#Converting both fields to the same data type
md['id'] = md['id'].astype(str)
nsfw_data['id'] = nsfw_data['id'].astype(str)
credits['id'] = credits['id'].astype(str)
md = md.merge(nsfw_data, on='id', how='left')
md = md.merge(credits, on='id', how='left')

In [15]:
md.shape[0]

45542

In [21]:
md = (md.drop(['budget'], axis=1)
         .join(md['budget'].apply(pd.to_numeric, errors='coerce')))



In [33]:
md = md.replace(0,'NaN')


In [34]:
fillInRevenue = md[md['revenue']=='NaN']

In [50]:
missingRev = fillInRevenue.imdb_id.astype(str).tolist()

In [51]:
# Pickling functions
def pickle_it(data, filename):
    with open(filename, "wb") as picklefile:
        pickle.dump(data, picklefile)

def load_pickle(filename):
    with open(filename, "rb") as picklefile: 
        return pickle.load(picklefile)

In [55]:
def get_imdb_soup(url):
    response = requests.get(url)
    page = response.text
    return BeautifulSoup(page) #for some reason cannot get attributes (ie.title,a,div) from prettify

def get_mv_urls(link_list):
    mv_url_list = []
    imdb = 'http://www.imdb.com/title/'
    for mvid in link_list:
        mv_url = imdb+mvid
        mv_url_list.append(mv_url)
    return mv_url_list
    pickle_it(mv_url_list,'mv_url_list.pkl')
    


In [78]:
def get_imdb_header(imdb_soup):
    imdb_header = imdb_soup.title
    return imdb_header


def get_imdb_title(imdb_header):
    header = imdb_header.text
    imdb_title = header.split('(')[0]
    try:
        return imdb_title
    except(ValueError,RuntimeError, TypeError, NameError):
        return None
def get_imdb_year(imdb_header):
    header = imdb_header.text
    imdb_year = header.split('(')[1]
    imdb_year
    year = re.findall(r'\d+', imdb_year)
    try:
        imdb_year = int(year[0])
        return imdb_year
    except(ValueError, RuntimeError, TypeError, NameError, IndexError):
        return None

def get_imdb_budget(soup):
        divs = [x.text for x in soup.find_all('div')]
        try:    
            for s in divs:
                start = 'Budget:'
                end = ' '
                result = re.findall('%s(.*)%s' % (start, end), s)
                for n in result:
                    regex = re.compile(r'\d+(?:,\d+)*')
                    bgt = re.findall(regex,n)
                    budget =[x.replace(',','') for x in bgt]
                    budint =[int(b) for b in budget]
                return budint[0]
        except(ValueError, RuntimeError, TypeError, NameError):
            return None
        
def get_imdb_gross(soup):
        divs = [x.text for x in soup.find_all('div')]
        for s in divs:
            start = 'Gross:'
            end = ' '
            result = re.findall('%s(.*)%s' % (start, end), s)
            for n in result:
                regex = re.compile(r'\d+(?:,\d+)*')
                grs = re.findall(regex,n)
                grss =[x.replace(',','') for x in grs]
                gross =[int(b) for b in grss]
        try:
            return gross[0]
        except:
            return None
                    


In [79]:
fullmovieList = get_mv_urls(missingRev)    

In [80]:
len(fullmovieList)

38108

In [85]:
mv_soup = get_imdb_soup(fullmovieList[21000])
mv_header = get_imdb_header(mv_soup)
mv_title = get_imdb_title(mv_header)
mv_year = get_imdb_year(mv_header)
mv_budget = get_imdb_budget(mv_soup)
mv_gross = get_imdb_gross(mv_soup)

print mv_header
print mv_title
print mv_year
print mv_budget
print mv_gross


<title>The Feminine Touch (1941) - IMDb</title>
The Feminine Touch 
1941
None
None


In [86]:
def get_mv_data(mv_id_list):
    mv_data_list = []
    count = 0
    headlabels = ['Title', 'Year', 'Budget','Gross']
    for mv_urlID in mv_id_list: #iterates through movie id urls
        mv_soup = get_imdb_soup(mv_urlID)
        mv_header = get_imdb_header(mv_soup)
        mv_title = get_imdb_title(mv_header)
        mv_year = get_imdb_year(mv_header)
        mv_budget = get_imdb_budget(mv_soup)
        mv_gross = get_imdb_gross(mv_soup)
        mv_data_dict = dict(zip(headlabels, [mv_title, mv_year, mv_budget, mv_gross]))
        count+=1
        mv_data_list.append(mv_data_dict)
    return mv_data_list

In [ ]:
mv_data = get_mv_data(fullmovieList[:500])

In [ ]:
print mv_data

In [ ]:
pickle_it(mv_url_list,'mv_data_list.pkl')